In [ ]:
#This Notebook is there to split retro and pro cohorts, 
# to create cases and controls 
# and to prepare for ML pipeline 

In [ ]:
import numpy as np
import pandas as pd
import sys

# Loading needed DataFrames

# load Case & Controls Complications MRNS

# unique are MRNs that have been diagnosed after HT_onset & for the first time
#Controls are the ones that have never had the disease
# all together CR= 59.606, Heart = 51.544, Renal = 59.950

All3_training = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/MRN_match/MRN_merged/All3_merged_training.pkl")
All3_test= pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/MRN_match/MRN_merged/All3_merged_test.pkl")

Cerebro_training = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/MRN_match/MRN_merged/Cerebro_merged_training.pkl")
Cerebro_test= pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/MRN_match/MRN_merged/Cerebro_merged_test.pkl")

Heart_training = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/MRN_match/MRN_merged/Heart_merged_training.pkl")
Heart_test= pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/MRN_match/MRN_merged/Heart_merged_test.pkl")

Renal_training = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/MRN_match/MRN_merged/Renal_merged_training.pkl")
Renal_test= pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/MRN_match/MRN_merged/Renal_merged_test.pkl")

All3_training = All3_training[['medical_record_number']]
All3_test = All3_test[['medical_record_number']]

Cerebro_training = Cerebro_training[['medical_record_number']]
Cerebro_test = Cerebro_test[['medical_record_number']]

Heart_training = Heart_training[['medical_record_number']]
Heart_test = Heart_test[['medical_record_number']]

Renal_training = Renal_training[['medical_record_number']]
Renal_test = Renal_test[['medical_record_number']]

All3_intersection = pd.concat([All3_training,All3_test])
Cerebro_intersection = pd.concat([Cerebro_training,Cerebro_test])
Heart_intersection = pd.concat([Heart_training,Heart_test])
Renal_intersection = pd.concat([Renal_training,Renal_test])

All3_intersection['medical_record_number']=All3_intersection['medical_record_number'].astype(str)
Cerebro_intersection['medical_record_number']=Cerebro_intersection['medical_record_number'].astype(str)
Heart_intersection['medical_record_number']=Heart_intersection['medical_record_number'].astype(str)
Renal_intersection['medical_record_number']=Renal_intersection['medical_record_number'].astype(str)

In [ ]:
import numpy as np
import pandas as pd
import sys

# Loading needed DataFrames

# load Case & Controls Complications MRNS

# unique are MRNs that have been diagnosed after HT_onset & for the first time
#Controls are the ones that have never had the disease
# all together CR= 59.606, Heart = 51.544, Renal = 59.950

All3 = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/All3_diseases/Cases_all3_cohorts_after_HT_onset.pkl")
All3_Controls = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/All3_diseases/Controls_No_all3_cohorts_Disease_Ever.pkl")

Cerebro = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Cerebrovascular_Diseases/Cases_CR_after_HT_onset.pkl")
Cerebro_Controls = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Cerebrovascular_Diseases/Controls_No_Cerebrovascular_Diseases_Ever.pkl')
   
Heart = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Heart_Diseases/Cases_heart_disease_after_HT_onset.pkl")
Heart_Controls = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Heart_Diseases/Controls_No_Heart_Diseases_Ever.pkl")

Renal = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Renal_Diseases/Cases_renal_disease_after_HT_onset.pkl")
Renal_Controls = pd.read_pickle("/home/kiwitn01/master_thesis_hypertension-complications/Cohort_Extraction/Complications/Renal_Diseases/Controls_No_Renal_Diseases_Ever.pkl")


df_final = pd.read_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Feature Extraction/Clean_Unsupervised_Dataframes/All_unsupervised_merged_clean_Diag_no_int_with_extra_BP.pkl')



In [ ]:
def prepare_df_for_split(df_Cases, df_Controls, df_final):
    
    df_Cases = df_Cases.reset_index('medical_record_number')
    df_Cases = df_Cases.sort_values('onset_year',ascending=True).drop_duplicates('medical_record_number', keep = 'first')
    df_Cases['Complication'] = '1'
    
    df_Controls = df_Controls[['HT_onset_year','date_of_birth_actual','age_in_days_HT_onset']]
    df_Controls['Complication'] = '0'
    df_Controls= df_Controls.reset_index('medical_record_number')
    
    Cohort = pd.concat([df_Cases, df_Controls], sort=True)
    Cohort= Cohort.set_index('medical_record_number')
    
    Train = Cohort.loc[Cohort['HT_onset_year'] < 2009]
    Test = Cohort.loc[Cohort['HT_onset_year'] >= 2009]
    
    Train['train_test'] = 'train'
    Test['train_test'] = 'test'
    
    Cohort = pd.concat([Train, Test], sort=True )
    
    df_final = df_final.set_index('medical_record_number')
    Cohort_Final = pd.merge(df_final, Cohort, on = 'medical_record_number', how = 'inner')
     
    return Cohort_Final    


In [ ]:
#setting the Cohort that we need

df_All3 = prepare_df_for_split(All3,All3_Controls, df_final)
df_Cerebro = prepare_df_for_split(Cerebro,Cerebro_Controls, df_final)
df_Heart = prepare_df_for_split(Heart,Heart_Controls, df_final)
df_Renal = prepare_df_for_split(Renal,Renal_Controls, df_final)

In [ ]:
df_All3 = df_All3.reset_index('medical_record_number')
df_Cerebro = df_Cerebro.reset_index('medical_record_number')
df_Heart = df_Heart.reset_index('medical_record_number')
df_Renal = df_Renal.reset_index('medical_record_number')

In [ ]:
df_All3

In [ ]:
df_merged_All3 = pd.merge(df_All3, All3_intersection, on = 'medical_record_number', how = 'inner')
df_merged_Cerebro = pd.merge(df_Cerebro, Cerebro_intersection, on = 'medical_record_number', how = 'inner')
df_merged_Heart = pd.merge(df_Heart, Heart_intersection, on = 'medical_record_number', how = 'inner')
df_merged_Renal = pd.merge(df_Renal, Renal_intersection, on = 'medical_record_number', how = 'inner')


In [ ]:
df_merged_All3.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Case_Control_Cohort_Creation/For_ML_Pipeline/Split_2009_for_genetic/All3.pkl')
df_merged_Cerebro.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Case_Control_Cohort_Creation/For_ML_Pipeline/Split_2009_for_genetic/Cerebro.pkl')
df_merged_Heart.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Case_Control_Cohort_Creation/For_ML_Pipeline/Split_2009_for_genetic/Heart.pkl')
df_merged_Renal.to_pickle('/home/kiwitn01/master_thesis_hypertension-complications/Case_Control_Cohort_Creation/For_ML_Pipeline/Split_2009_for_genetic/Renal.pkl')


In [ ]:
df_merged_Renal.loc[(df_merged_Renal['train_test'] == 'train') & (df_merged_Renal['Complication'] == '0')]
#df_merged.loc[df_merged['train_test'] == 'test']